# LSTM with Word Embedding

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import re
import logging
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer, one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Flatten, Embedding, LSTM, GRU, Dropout
from tensorflow.keras.models import Sequential

# Reading datasets

In [6]:
data  = pd.read_csv('Data_processed/dataset.csv',sep=',',names=['Msg','Tag'])
data1 = pd.read_csv('Data_processed/dataset_POS.csv',sep=',',names=['Msg','Tag'])
data2 = pd.read_csv('Data_processed/dataset_stemmed.csv',sep=',',names=['Msg','Tag'])

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10944 entries, 0 to 10943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Msg     10938 non-null  object
 1   Tag     10944 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 171.1+ KB


In [8]:
data.dropna(inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10938 entries, 0 to 10943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Msg     10938 non-null  object
 1   Tag     10938 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 256.4+ KB


In [9]:
data.head()

,Msg,Tag
0,The thing disgusting White woman groid White w...,1
1,Americans acting like know talking,0
2,Also intrested check webpage info european ame...,0
3,I think need take stand homes across country a...,0
4,I think connection homosexuality Christianity ...,1


In [10]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10944 entries, 0 to 10943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Msg     10764 non-null  object
 1   Tag     10944 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 171.1+ KB


In [11]:
data1.dropna(inplace=True)
data1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10764 entries, 0 to 10943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Msg     10764 non-null  object
 1   Tag     10764 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 252.3+ KB


In [12]:
data1.head()

,Msg,Tag
0,TheDT thingNN disgustingVBG womanNN groidNN wo...,1
1,AmericansNNPS actingVBG likeIN knowNNS talkingVBG,0
2,AlsoRB intrestedVBN checkNN webpageNN infoJJ e...,0
3,IPRP thinkVBP needJJ takeVBP standVBP homesNNS...,0
4,IPRP thinkVBP connectionNN homosexualityNN cou...,1


In [13]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10944 entries, 0 to 10943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Msg     10931 non-null  object
 1   Tag     10944 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 171.1+ KB


In [14]:
data2.dropna(inplace=True)
data2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10931 entries, 0 to 10943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Msg     10931 non-null  object
 1   Tag     10931 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 256.2+ KB


In [15]:
data2.head()

,Msg,Tag
0,theDT thingNN disgustJJ whiteJJ womanNN groidJ...,1
1,americanJJ actNN likeIN knowJJ talkNN,0
2,alsoRB intrestRB checkVB webpagNN infoJJ europ...,0
3,IPRP thinkVBP needJJ takeVBP standVBP homeNN a...,0
4,IPRP thinkVBP connectJJ homosexuNN christianJJ...,1


In [16]:
data_x=data["Msg"]
data_y=data["Tag"]

data1_x=data1["Msg"]
data1_y=data1["Tag"]

data2_x=data2["Msg"]
data2_y=data2["Tag"]

In [17]:
vocab_size = 10000

In [18]:
data_x_n = data_x.to_numpy()
data_x_n

array(['The thing disgusting White woman groid White woman drags White child filth ',
       'Americans acting like know talking ',
       'Also intrested check webpage info european american town building ',
       ..., 'No truth 88WHITE POWERWHITE VICTORYWHITE PRIDE88',
       '4 cyclinder motorcycle Historic vehicle Laurin Klement T B 18991903 YouTube Historic vehicle Torpedo V4 1909 YouTube Historic vehicle Torpedo V4 1909 httpthekneeslidercomimages2012rightsidejpg Handlebar camera mount ride video ',
       'Hi I thought I leave note wish Southern Gentlemen Ladies happy Robert Edward Lee day '],
      dtype=object)

In [26]:
data1_x_n = data1_x.to_numpy()
data1_x_n

array(['TheDT thingNN disgustingVBG womanNN groidNN womanNN dragsVBZ childNN filthNN',
       'AmericansNNPS actingVBG likeIN knowNNS talkingVBG',
       'AlsoRB intrestedVBN checkNN webpageNN infoJJ europeanJJ americanJJ townNN buildingNN',
       ..., 'NoDT truthNN 88WHITECD',
       '4CD cyclinderNN motorcycleNN vehicleNN 18991903CD vehicleNN 1909CD vehicleNN 1909CD httpthekneeslidercomimages2012rightsidejpgNN cameraNN mountNN rideNN videoNN',
       'IPRP thoughtVBD IPRP leaveVBP noteJJ wishJJ happyJJ dayNN'],
      dtype=object)

In [25]:
data2_x_n = data2_x.to_numpy()
data2_x_n

array(['theDT thingNN disgustJJ whiteJJ womanNN groidJJ whiteJJ womanNN dragVBZ whiteJJ childNN filthNN',
       'americanJJ actNN likeIN knowJJ talkNN',
       'alsoRB intrestRB checkVB webpagNN infoJJ europeanJJ americanJJ townNN buildNN',
       ..., 'NoDT truthNN 88whiteCD powerwhitNN victorywhitNN pride88NN',
       '4CD cyclindNN motorcyclNN historNN vehiclNN laurinJJ klementNN 18991903CD youtubNN historNN vehiclNN torpedoNN 1909CD youtubNN historNN vehiclNN torpedoNN 1909CD httpthekneeslidercomimages2012rightsidejpgNN handlebarNN cameraNN mountNN rideNN videoNN',
       'IPRP thoughtVBD IPRP leavVBP noteJJ wishJJ southernJJ gentlemenNNS ladiVBP happiNN robertNN edwardJJ leeNN dayNN'],
      dtype=object)

# Maximum Sentence Length

In [48]:
def max_sen_length(data):
    mx_len = 0
    for sen in data:
        words = sen.split()
        if len(words) > mx_len:
            mx_len = len(words)
    return mx_len

In [49]:
sen_len = max_sen_length(data_x_n)

In [50]:
sen_len1 = max_sen_length(data1_x_n)

In [51]:
sen_len2 = max_sen_length(data2_x_n)

# One Hot Representation

In [52]:
onehot_enc = [one_hot(sen, vocab_size) for sen in data_x_n]

In [53]:
onehot1_enc = [one_hot(sen, vocab_size) for sen in data1_x_n]

In [54]:
onehot2_enc = [one_hot(sen, vocab_size) for sen in data2_x_n]

# Embedding Representation

In [58]:
embed_repr = pad_sequences(onehot_enc, padding='pre', maxlen=sen_len)

In [59]:
embed_repr1 = pad_sequences(onehot1_enc, padding='pre', maxlen=sen_len1)

In [60]:
embed_repr2 = pad_sequences(onehot2_enc, padding='pre', maxlen=sen_len2)

In [61]:
print(embed_repr[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 7817
 8679 4890 8613 4711 8244 8613 4711 3163 8613 3018 4066]


In [62]:
print(embed_repr1[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0 8394 3607  988 2817 9734
 2817 5899 6402 5190]


In [64]:
print(embed_repr2[0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0 8394 3607 4586 9074 2817  872 9074 2817 7880
 9074 6402 5190]


#  LSTM Model Creation

In [65]:
embedding_vector_features = 100

In [67]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_vector_features,input_length=sen_len))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 151, 100)          1000000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None


In [68]:
model1 = Sequential()
model1.add(Embedding(vocab_size, embedding_vector_features,input_length=sen_len1))
model1.add(LSTM(100))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 144, 100)          1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None


In [69]:
model2 = Sequential()
model2.add(Embedding(vocab_size, embedding_vector_features,input_length=sen_len2))
model2.add(LSTM(100))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
print(model2.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 143, 100)          1000000   
_________________________________________________________________
lstm_3 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None


# Train Test Split

In [70]:
x_train, x_test, y_train, y_test = train_test_split(embed_repr, data_y, test_size=0.2, random_state=4)
x1_train, x1_test, y1_train, y1_test = train_test_split(embed_repr1, data1_y, test_size=0.2, random_state=4)
x2_train, x2_test, y2_train, y2_test = train_test_split(embed_repr2, data2_y, test_size=0.2, random_state=4)

# Model Training

In [71]:
model.fit(x_train,y_train, validation_data=(x_test,y_test),epochs=10,batch_size=64)

Train on 8750 samples, validate on 2188 samples
Epoch 1/10
8750/8750 [==============================] - 25s 3ms/sample - loss: 0.3764 - acc: 0.8669 - val_loss: 0.3220 - val_acc: 0.8771
Epoch 2/10
8750/8750 [==============================] - 25s 3ms/sample - loss: 0.2332 - acc: 0.9078 - val_loss: 0.3502 - val_acc: 0.8688
Epoch 3/10
8750/8750 [==============================] - 25s 3ms/sample - loss: 0.1445 - acc: 0.9474 - val_loss: 0.4525 - val_acc: 0.8624
Epoch 4/10
8750/8750 [==============================] - 26s 3ms/sample - loss: 0.1013 - acc: 0.9645 - val_loss: 0.5174 - val_acc: 0.8492
Epoch 5/10
8750/8750 [==============================] - 24s 3ms/sample - loss: 0.0659 - acc: 0.9774 - val_loss: 0.6228 - val_acc: 0.8195
Epoch 6/10
8750/8750 [==============================] - 23s 3ms/sample - loss: 0.0431 - acc: 0.9871 - val_loss: 0.8160 - val_acc: 0.8355
Epoch 7/10
8750/8750 [==============================] - 23s 3ms/sample - loss: 0.2355 - acc: 0.9090 - val_loss: 0.4338 - val_acc: 

In [72]:
model1.fit(x1_train,y1_train, validation_data=(x1_test,y1_test),epochs=10,batch_size=64)

Train on 8611 samples, validate on 2153 samples
Epoch 1/10
8611/8611 [==============================] - 24s 3ms/sample - loss: 0.3846 - acc: 0.8671 - val_loss: 0.3627 - val_acc: 0.8611
Epoch 2/10
8611/8611 [==============================] - 23s 3ms/sample - loss: 0.2497 - acc: 0.9008 - val_loss: 0.3801 - val_acc: 0.8583
Epoch 3/10
8611/8611 [==============================] - 23s 3ms/sample - loss: 0.1443 - acc: 0.9484 - val_loss: 0.5307 - val_acc: 0.8504
Epoch 4/10
8611/8611 [==============================] - 24s 3ms/sample - loss: 0.0836 - acc: 0.9704 - val_loss: 0.6451 - val_acc: 0.8198
Epoch 5/10
8611/8611 [==============================] - 23s 3ms/sample - loss: 0.0538 - acc: 0.9828 - val_loss: 0.7573 - val_acc: 0.8323
Epoch 6/10
8611/8611 [==============================] - 22s 3ms/sample - loss: 0.0270 - acc: 0.9919 - val_loss: 0.9323 - val_acc: 0.8045
Epoch 7/10
8611/8611 [==============================] - 24s 3ms/sample - loss: 0.0174 - acc: 0.9948 - val_loss: 0.9952 - val_acc: 

In [73]:
model2.fit(x2_train,y2_train, validation_data=(x2_test,y2_test),epochs=10,batch_size=64)

Train on 8744 samples, validate on 2187 samples
Epoch 1/10
8744/8744 [==============================] - 26s 3ms/sample - loss: 0.3919 - acc: 0.8688 - val_loss: 0.3564 - val_acc: 0.8637
Epoch 2/10
8744/8744 [==============================] - 25s 3ms/sample - loss: 0.2472 - acc: 0.9035 - val_loss: 0.3772 - val_acc: 0.8519
Epoch 3/10
8744/8744 [==============================] - 25s 3ms/sample - loss: 0.1453 - acc: 0.9453 - val_loss: 0.4617 - val_acc: 0.8377
Epoch 4/10
8744/8744 [==============================] - 26s 3ms/sample - loss: 0.0866 - acc: 0.9718 - val_loss: 0.6199 - val_acc: 0.8441
Epoch 5/10
8744/8744 [==============================] - 26s 3ms/sample - loss: 0.0538 - acc: 0.9809 - val_loss: 0.7302 - val_acc: 0.8487
Epoch 6/10
8744/8744 [==============================] - 27s 3ms/sample - loss: 0.1641 - acc: 0.9499 - val_loss: 0.5480 - val_acc: 0.8400
Epoch 7/10
8744/8744 [==============================] - 26s 3ms/sample - loss: 0.0669 - acc: 0.9762 - val_loss: 0.6704 - val_acc: 

# Prediction and Model Accuracy (Without POS and Stemming)

In [74]:
y_pred = model.predict_classes(x_test)

In [75]:
confusion_matrix(y_test, y_pred)

array([[1749,  166],
       [ 187,   86]])

In [76]:
accuracy_score(y_test, y_pred)

0.8386654478976234

# Prediction and Model Accuracy (POS)

In [77]:
y1_pred = model1.predict_classes(x1_test)

In [78]:
confusion_matrix(y1_test, y1_pred)

array([[1674,  177],
       [ 224,   78]])

In [79]:
accuracy_score(y1_test, y1_pred)

0.8137482582443103

# Prediction and Model Accuracy (Stemming + POS)

In [80]:
y2_pred = model2.predict_classes(x2_test)

In [81]:
confusion_matrix(y2_test, y2_pred)

array([[1772,  114],
       [ 224,   77]])

In [82]:
accuracy_score(y2_test, y2_pred)

0.8454503886602652